# 14장 인공지능과 웹 서비스 연결

## 14.1 날씨 정보 가져오기

### 14.1.1 날씨 제공 서비스의 웹 API 키 생성

### 14.1.2 Weather API 이용 방법

### 14.1.3 Weather API를 활용해 날씨 정보 가져오기

[14장: 354페이지]

In [ ]:
import requests
import os

WEATHER_API_KEY = os.environ["WEATHER_API_KEY"] # API 키 지정
city = "Seoul" # 도시는 서울로 지정
lang_code = "ko"

url = "http://api.weatherapi.com/v1/current.json"
parameters = {"key":WEATHER_API_KEY, "q":city, "lang":lang_code}

r = requests.get(url, params=parameters)
current_weather = r.json()
print(current_weather)

[14장: 355페이지]

In [ ]:
name = current_weather['location']['name']
localtime = current_weather['location']['localtime']
temp_c = current_weather['current']['temp_c']
temp_f = current_weather['current']['temp_f']
condition_text = current_weather['current']['condition']['text']

print(f"현재({localtime}) {name}의 날씨 정보: {condition_text}, 섭씨 {temp_c}도, 화씨 {temp_f}도")

In [ ]:
import requests
import os
import json
import re

def get_current_weather(location, unit="섭씨"):
    
    reg = re.compile(r'[a-zA-Z]') # 영어 입력인지를 검사하는 정규식  
    
    if reg.match(location): # 영어로 도시 이름을 지정한 경우  
        city = location # 영어 도시 이름을 바로 지정
    else: # 영어로 지정하지 않은 경우 
        city_names = {"서울": "Seoul", "인천": "Incheon", "대전": "Daejeon", 
                      "대구": "Daegu", "부산": "Busan", "광주": "Gwangju",
                      "수원": "Suwon", "파리": "Paris", "뉴욕": "New York"}
        city = city_names[location] # 한글 도시 이름을 영어로 변경
    
    WEATHER_API_KEY = os.environ["WEATHER_API_KEY"] # API 키 지정

    url = "http://api.weatherapi.com/v1/current.json"
    parameters = {"key":WEATHER_API_KEY, "q":city}

    r = requests.get(url, params=parameters)
    current_weather = r.json()
    
    name = current_weather['location']['name'] # 설정 지역
    localtime = current_weather['location']['localtime'] # 날짜 및 시각
    temp_c = current_weather['current']['temp_c'] # 섭씨 온도
    temp_f = current_weather['current']['temp_f'] # 화씨 온도
    condition_text = current_weather['current']['condition']['text'] # 날씨 상태
     
    # unit 지정에 따라서 섭씨 온도 혹은 화씨 온도를 지정
    if unit == "섭씨":
        temp = temp_c
    elif unit == "화씨":
        temp = temp_f
    else:
        unit == "섭씨"
        temp = temp_c
        
    weather_info = {
            "location": name,
            "temperature": temp,
            "unit": unit,
            "current weather": condition_text,
            "local time": localtime
    }
    
    return json.dumps(weather_info, ensure_ascii=False) # JSON 형식으로 반환

[14장: 357페이지]

In [ ]:
get_current_weather('서울')

In [ ]:
get_current_weather('파리')

### 14.1.4 OpenAI의 함수 호출로 날씨 정보 가져오기

[14장: 358페이지]

In [ ]:
import openai
import os

# API 키 설정
openai.api_key = os.environ["OPENAI_API_KEY"]

# Chat Completions API를 이용해 사용자 입력에 따라 함수를 호출하고 응답하는 함수
def run_conversation(user_query):
    # 사용자 입력
    messages = [{"role": "user", "content": user_query}] 
        
    # 함수 정보 입력   
    functions = [                                        
        {
            "name": "get_current_weather",
            "description": "도시 이름을 입력해 현재 날씨, 날짜, 시각, 몇 시인지 가져오기",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "도시 이름, 예를 들면, 서울, 부산, 대전",
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["섭씨", "화씨"],
                        "description": "온도 단위로 섭씨 혹은 화씨",
                    },
                },
                "required": ["location"], # 필수 입력 변수 지정
            }
        }        
    ]
    # 1단계: 사용자 입력과 함수 정보를 Chat Completions API 모델로 보내기    
    response = openai.ChatCompletion.create( # Chat Completions API 모델로 보내기
            # model="gpt-3.5-turbo",
            model="gpt-4",
            messages=messages,
            functions=functions,
            function_call="auto"
    )
    # 2단계: 응답 생성
    response_message = response["choices"][0]["message"] # 모델의 응답 메시지
    
    if response_message.get("function_call"): # 응답이 함수 호출인지 확인하기
        # 3단계: JSON 객체를 분석해 함수 이름과 인수를 추출한 후에 함수 호출
        # (주의: JSON 응답이 항상 유효하지 않을 수 있음)
        
        # 호출할 함수 이름을 지정 
        # (아래는 하나의 함수를 지정했지만 여러 함수 지정 가능)
        available_functions = {"get_current_weather": get_current_weather}     

        # 함수 이름 추출
        function_name = response_message["function_call"]["name"]
        
        # 호출할 함수 선택
        fuction_to_call = available_functions[function_name]
        
        # 함수 호출을 위한 인수 추출
        function_args = json.loads(response_message["function_call"]["arguments"])
        
        # 함수 호출 및 반환 결과 받기
        function_response = fuction_to_call(
            location=function_args.get("location"), # 인수 지정
            unit=function_args.get("unit")
        )
        
        print("[호출한 함수의 응답 결과]\n", function_response)
        
        # 4단계: 함수 호출 결과를 기존 메시지에 추가하고,
        #        Chat Completions API 모델로 보내 응답받기

        # 함수 호출 결과를 기존 메시지에 추가하기
        messages.append(response_message)  # 기존 messages에 조력자 응답 추가
        messages.append(                   # 함수와 함수 호출 결과 추가
            {
                "role": "function",           # roll: function으로 지정
                "name": function_name,        # name: 호출할 함수 이름 지정
                "content": function_response, # content: 함수 호출 결과 지정
            }
        )
        # 함수 호출 결과를 추가한 메시지를 Chat Completions API 모델로 보내 응답받기
        second_response = openai.ChatCompletion.create(
            # model="gpt-3.5-turbo",
            model="gpt-4",
            messages=messages,
        ) 
        return second_response # 두 번째 응답 반환
    
    return response_message # 응답 메시지 반환

[14장: 360페이지]

In [ ]:
user_query = "현재 수원의 날씨는 어떠한가요?"
response = run_conversation(user_query)
response_content = response["choices"][0]["message"]["content"]

print("[최종 응답 결과]\n", response_content)

[14장: 361페이지]

In [ ]:
json.dumps(response, ensure_ascii=False)

In [ ]:
user_query = "지금 뉴욕의 날씨는?"
response = run_conversation(user_query)
response_content = response["choices"][0]["message"]["content"]

print("[최종 응답 결과]\n", response_content)

In [ ]:
user_query = "현재 뉴욕의 날씨는? 온도는 화씨로 표시"
response = run_conversation(user_query)
response_content = response["choices"][0]["message"]["content"]

print("[최종 응답 결과]\n", response_content)

[14장: 362페이지]

In [ ]:
user_query = "서울의 현재 시각은?"
response = run_conversation(user_query)
response_content = response["choices"][0]["message"]["content"]

print("[최종 응답 결과]\n", response_content)

[14장: 362페이지]

In [ ]:
user_query = "파리는 지금 몇 시?"
response = run_conversation(user_query)
response_content = response["choices"][0]["message"]["content"]

print("[최종 응답 결과]\n", response_content)

## 14.2 이메일 보내기

### 14.2.1 이메일을 보내기 위한 사전 준비

### 14.2.2 파이썬 코드로 이메일 보내기

[14장: 371페이지]

In [ ]:
import smtplib
from email.message import EmailMessage
 
option = "gmail" # 구글 G메일 이용
# option = "nmail" # 네이버 메일 이용

if option=="gmail": # 구글 G메일의 SMTP 서버 이용    
    smtp_server = "smtp.gmail.com" # 구글 G메일의 SMTP 서버 주소
    port = 587 # TLS 암호화 프로토콜 사용하는 포트

    email_address = "본인_ID@gmail.com" # 자신의 이메일 주소 입력
    password = "qazwsxedcrfvtgby"       # 앱 비밀번호(16자리) 입력

elif option=="nmail": # 네이버 메일의 SMTP 서버 이용
    smtp_server = "smtp.naver.com" # 네이버 메일의 SMTP 서버 주소
    port = 587 # TLS 암호화 프로토콜 사용하는 포트
    
    email_address = "본인_ID@naver.com" # 자신의 이메일 주소 입력
    password = "qwerty12345"            # 자신의 네이버 로그인 비밀번호

smtp = smtplib.SMTP(smtp_server, port) # SMTP 객체를 생성
smtp.starttls()
smtp.login(email_address, password) # SMTP 서버에 로그인

# 이메일 제목, 수신자 이메일 주소, 본문의 내용을 지정
subject = "테스트 이메일 보내기" # 이메일 제목
from_address = email_address     # 송신자 이메일 주소 
to_address = "to@example.com"    # 수신자 이메일 주소
body = "SMTP를 이용해 이메일을 보내는 테스트 본문입니다." # 본문

msg = EmailMessage() # 이메일 메시지 객체 생성
msg['Subject'] = subject # 이메일 제목
msg['From'] = from_address   # 수신자 이메일 주소
msg['To'] = to_address   # 수신자 이메일 주소
msg.set_content(body)    # 이메일 본문

smtp.send_message(msg) # 메시지 보내기
smtp.quit() # SMTP 세션을 종료

[14장: 372페이지]

In [ ]:
import smtplib

# 이메일 전송을 위한 SMTP 서버에 로그인
def smtp_setting_login(option="gmail"):
    
    if option=="gmail": # 구글 G메일의 SMTP 서버 이용            
        smtp_server = "smtp.gmail.com" # 구글 G메일의 SMTP 서버 주소
        port = 587 # TLS 암호화 프로토콜 사용하는 포트

        email_address = "본인_ID@gmail.com" # 자신의 이메일 주소 입력
        password = "qazwsxedcrfvtgby"       # 앱 비밀번호(16자리) 입력

    elif option=="nmail": # 네이버 메일의 SMTP 서버 이용        
        smtp_server = "smtp.naver.com" # 네이버 메일의 SMTP 서버 주소
        port = 587 # TLS 암호화 프로토콜 사용하는 포트

        email_address = "본인_ID@naver.com" # 자신의 이메일 주소 입력
        password = "qwerty12345"            # 자신의 네이버 로그인 비밀번호
    
    smtp = smtplib.SMTP(smtp_server, port) # SMTP 객체를 생성
    smtp.starttls()
    smtp.login(email_address, password) # SMTP 서버에 로그인  
    
    return smtp, email_address # SMTP 객체와 로그인 이메일 주소 반환

[14장: 373페이지]

In [ ]:
import smtplib
from email.message import EmailMessage
import json

# 이름과 본문 내용으로 이메일을 보내는 함수    
def send_email(to, body): # to: 수신자 이름, body: 이메일 본문
    
    # Chat Completions API가 수신자_이름@example.com 과 같이
    # 생성하는 것에 대비해 수신자 이름만 추출
    to = to.split('@')[0] 

    # 수신자 이름과 이메일 주소를 지정한 딕셔너리 변수
    address_book = {"박연진":"Park123@example.com", 
                    "전재준":"Jeon456@example.com",
                    "이사라":"Lee789@example.com",
                    "고교동창_최혜정":"Choi123@example.com",
                    "손명오":"Son456@example.com",
                    "주여정":"Joo789@example.com"}
       
    # 이메일 전송을 위한 SMTP 서버에 로그인
    smtp, email_address = smtp_setting_login("gmail")   # 구글 G메일 이용
    # smtp, email_address = smtp_setting_login("nmail") # 네이버 메일 이용
    
    subject = "OpenAI의 함수 호출로 보낸 이메일" # 이메일 제목
    from_address = email_address # 송신자 이메일 주소 
    to_address = email_address   # 수신자 이메일 주소을 가져오기 전에 송신자 이메일 주소로 초기화

    # address_book의 어떤 키가 to의 이름을 전체 혹은 부분적으로 포함하면 이메일 주소를 가져옴
    for dict_key in address_book.keys(): # address_book의 모든 키에 대해 반복문을 수행
        if to in dict_key:          # 해당 키가 to의 이름을 전체 혹은 부분적으로 포함하면
            to_address = address_book.get(dict_key) # 해당 키로 이메일 주소를 얻음
            break  # 이메일 주소를 얻었으므로, 반복문을 빠져나옴

    msg = EmailMessage()        # 이메일 메시지 객체 생성
    msg['Subject'] = subject    # 이메일 제목 지정
    msg['From'] = from_address  # 송신자 이메일 주소 
    msg['To'] = to_address      # 수신자 이메일 주소 
    msg.set_content(body)       # 이메일 본문

    smtp.send_message(msg) # 메시지 보내기
    smtp.quit() # SMTP 세션을 종료
    
    send_mail = {
        "from": "문동은",
        "to": to,
        "body": body
    }
    
    return json.dumps(send_mail, ensure_ascii=False) # JSON 형식으로 반환

[14장: 375페이지]

In [ ]:
send_email("박연진", "내일 10시에 거기서 만나.")

### 14.2.3 OpenAI의 함수 호출로 이메일 보내기

[14장: 376페이지]

In [ ]:
import openai
import os

# API 키 설정
openai.api_key = os.environ["OPENAI_API_KEY"]

# Chat Completions API를 이용해 사용자 입력에 따라 함수를 호출하고 응답하는 함수
def run_conversation_for_email(user_query):
    # 사용자 입력
    messages = [{"role": "user", "content": user_query}] 
        
    # 함수 정보 입력   
    functions = [                                        
        {
            "name": "send_email",
            "description": "이름과 본문을 지정해 이메일 보내기, ~에게 이메일 보내기",
            "parameters": {
                "type": "object",
                "properties": {
                    "to": {
                        "type": "string",
                        "description": "사람 이름, 친구 이름, 받는 사람 이름",
                    },
                    "body": {
                        "type": "string",
                        "description": "이메일 본문",
                    },
                },
                "required": ["to", "body"], # 필수 입력 변수 지정
            }
        }        
    ]
    # 1단계: 사용자 입력과 함수 정보를 Chat Completions API 모델로 보내기    
    response = openai.ChatCompletion.create( # Chat Completions API 모델로 보내기
            # model="gpt-3.5-turbo", # gpt-3.5-turbo 모델
            model="gpt-4",           # gpt-4 모델
            messages=messages,
            functions=functions,
            function_call="auto"
    )
    # 2단계: 응답 생성
    response_message = response["choices"][0]["message"] # 모델의 응답 메시지
    
    if response_message.get("function_call"): # 응답이 함수 호출인지 확인하기
        # 3단계: JSON 객체를 분석해 함수 이름과 인수를 추출한 후에 함수 호출
        # (주의: JSON 응답이 항상 유효하지 않을 수 있음)
        
        # 호출할 함수 이름을 지정 
        # (아래는 하나의 함수를 지정했지만 여러 함수 지정 가능)
        available_functions = {"send_email": send_email}     

        # 함수 이름 추출
        function_name = response_message["function_call"]["name"]
        
        # 호출할 함수 선택
        fuction_to_call = available_functions[function_name]
        
        # 함수 호출을 위한 인수 추출
        function_args = json.loads(response_message["function_call"]["arguments"])
        
        # 함수 호출 및 반환 결과 받기
        function_response = fuction_to_call(
            to=function_args.get("to"), # 인수 지정
            body=function_args.get("body")
        )
        
        print("[호출한 함수의 응답 결과]\n", function_response)
        
        # 4단계: 함수 호출 결과를 기존 메시지에 추가하고,
        #        Chat Completions API 모델로 보내 응답받기

        # 함수 호출 결과를 기존 메시지에 추가하기
        messages.append(response_message)  # 기존 messages에 조력자 응답 추가
        messages.append(                   # 함수와 함수 호출 결과 추가
            {
                "role": "function",           # roll: function으로 지정
                "name": function_name,        # name: 호출할 함수 이름 지정
                "content": function_response, # content: 함수 호출 결과 지정
            }
        )
        # 함수 호출 결과를 추가한 메시지를 Chat Completions API 모델로 보내 응답받기
        second_response = openai.ChatCompletion.create(
            # model="gpt-3.5-turbo",  # gpt-3.5-turbo 모델
            model="gpt-4",            # gpt-4 모델
            messages=messages,
        ) 
        return second_response # 두 번째 응답 반환
    
    return response_message # 응답 메시지 반환

[14장: 378페이지]

In [ ]:
user_query = "박연진에게 내일 10시에 만나자고 이메일 보내줘"
response = run_conversation_for_email(user_query)

try: # 함수 호출을 정상적으로 수행한 경우
    response_content = response["choices"][0]["message"]["content"]
    print("[최종 응답 결과]\n", response_content)
except: # 함수 호출을 하지 못한 경우
    print("함수 호출을 하지 못했습니다.")
    print("[응답 내용]", response['content'])

[14장: 379페이지]

In [ ]:
user_query = "전재준에게 약속 장소 알려달라는 이메일 보내줘"
response = run_conversation_for_email(user_query)

try:
    response_content = response["choices"][0]["message"]["content"]
    print("[최종 응답 결과]\n", response_content)
except:
    print("함수 호출을 하지 못했습니다.")
    print("[응답 내용]", response['content'])

[14장: 380페이지]

In [ ]:
user_query = "이사라에게 이메일 보내줘"
response = run_conversation_for_email(user_query)

try:
    response_content = response["choices"][0]["message"]["content"]
    print("[최종 응답 결과]\n", response_content)
except:
    print("함수 호출을 하지 못했습니다.")
    print("[응답 내용]", response['content'])

In [ ]:
user_query = "내일 오후 1시에 만나자고 이메일 보내줘"
response = run_conversation_for_email(user_query)

try:
    response_content = response["choices"][0]["message"]["content"]
    print("[최종 응답 결과]\n", response_content)
except:
    print("함수 호출을 하지 못했습니다.")
    print("[응답 내용]", response['content'])

[14장: 381페이지]

In [ ]:
user_query = "최혜정에게 내일 오후 1시에 만나자고 이메일 보내줘"
response = run_conversation_for_email(user_query)

try:
    response_content = response["choices"][0]["message"]["content"]
    print("[최종 응답 결과]\n", response_content)
except:
    print("함수 호출을 하지 못했습니다.")
    print("[응답 내용]", response['content'])

In [ ]:
user_query = "'고교동창_최혜정'에게 내일 오후 1시에 만나자고 이메일 보내줘"
response = run_conversation_for_email(user_query)

try:
    response_content = response["choices"][0]["message"]["content"]
    print("[최종 응답 결과]\n", response_content)
except:
    print("함수 호출을 하지 못했습니다.")
    print("[응답 내용]", response['content'])

[14장: 382페이지]

In [ ]:
user_query = "손명호님에게 그간 작성한 신규 사업에 대한 보고서의 진행 상황을 문의하는 이메일 보내줘"
response = run_conversation_for_email(user_query)

try:
    response_content = response["choices"][0]["message"]["content"]
    print("[최종 응답 결과]\n", response_content)
except:
    print("함수 호출을 하지 못했습니다.")
    print("[응답 내용]", response['content'])

[14장: 383페이지]

In [ ]:
user_query = "주여정에게 '여정아.\n\n 지난번에 이야기했던 계획 바로 진행시켜.'라고 이메일 보내줘"
response = run_conversation_for_email(user_query)
response_content = response["choices"][0]["message"]["content"]

print("[최종 응답 결과]\n", response_content)

## 14.3 정리